In [12]:
#Import libraries for statistical, plot analysis and regression 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

In [2]:
# Load the Lipper TASS hedge fund data
TASS = pd.read_excel("C:/Master Thesis final/TASS_data.xlsx")
TASS.head()

,ref,refdate,monthly_date,TimeIndicator,ROR,AUM_Filled,T_Bill,downside,upside,currencycode,...,date,month_of_quarter,month,Net cash flow,style_return,st_adj_rateofreturn,DeltaMgr,DeltaOwn,TotalDelta,TotalDelta_Winso_01
0,5,1993-01-31,1993-01-01,192,-0.035175,NaN,0.002500,0.001419,0.000000,USD,...,1993-01-31 00:00:00,1,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,5,1993-02-28,1993-02-01,193,-0.050656,NaN,0.002442,0.002819,0.000000,USD,...,1993-02-28 00:00:00,2,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,5,1993-03-31,1993-03-01,194,0.022073,NaN,0.002458,0.000000,0.019615,USD,...,1993-03-31 00:00:00,3,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,5,1993-04-30,1993-04-01,195,0.087431,NaN,0.002392,0.000000,0.085039,USD,...,1993-04-30 00:00:00,1,4,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1993-05-31,1993-05-01,196,0.079997,NaN,0.002467,0.000000,0.077530,USD,...,1993-05-31 00:00:00,2,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Forward and Backward fill imputation technique for missing values
# Mean imputation technique has not been used to avoid bias and skewed results
# The target variable is processed first, to continue to build the database hereafter based on this support variable 
returns_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'ROR')
returns_df.ffill().bfill()

ref,5,8,9,10,11,18,23,26,31,35,...,105709,105710,105756,105785,105832,105870,106079,106080,106103,106134
refdate,,,,,,,,,,,,,,,,,,,,,
1993-01-31,-0.035175,0.049116,0.185567,0.045809,0.044878,-0.1790,0.0605,0.00637,0.0294,0.0260,...,0.007131,0.006097,-0.00757,0.0601,-0.0166,0.01640,-0.0034,0.00225,0.00084,-0.005620
1993-02-28,-0.050656,0.010300,0.050593,0.019571,0.022409,-0.1790,0.0605,0.00637,0.0294,0.0120,...,0.007131,0.006097,-0.00757,0.0601,-0.0166,0.01640,-0.0034,0.00225,0.00084,-0.005620
1993-03-31,0.022073,0.009268,0.047404,0.040219,0.052968,-0.1790,0.0605,0.00637,0.0294,-0.0060,...,0.007131,0.006097,-0.00757,0.0601,-0.0166,0.01640,-0.0034,0.00225,0.00084,-0.005620
1993-04-30,0.087431,0.013774,0.040948,0.028120,0.033825,-0.1790,0.0605,0.02033,0.0294,0.0200,...,0.007131,0.006097,-0.00757,0.0601,-0.0166,0.01640,-0.0034,0.00225,0.00084,-0.005620
1993-05-31,0.079997,0.040761,0.058661,0.032479,0.028523,-0.1790,0.0605,0.02008,0.0294,0.0230,...,0.007131,0.006097,-0.00757,0.0601,-0.0166,0.01640,-0.0034,0.00225,0.00084,-0.005620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-05-31,-0.072000,0.005688,-0.049829,0.038722,0.063176,0.0543,0.0209,-0.00030,0.0196,0.0039,...,0.004538,0.005178,-0.00757,0.0601,0.0000,0.01239,-0.0034,-0.00286,-0.04040,-0.008213
2015-06-30,-0.072000,0.005688,-0.049829,0.038722,0.063176,0.0543,0.0209,-0.00030,0.0196,-0.0132,...,0.004538,0.005178,0.00000,0.0204,0.0000,-0.09222,-0.0034,0.00000,-0.01470,-0.005976
2015-07-31,-0.072000,0.005688,-0.049829,0.038722,0.063176,0.0543,0.0209,-0.00030,0.0196,-0.0127,...,0.004538,0.005178,0.00000,0.0024,0.0000,0.09483,-0.0034,0.00000,-0.00010,-0.023580


In [4]:
# All the remaining columns have been processed to fill null values, either with forward backward fill or mean imputation
''' Additionally each variable is split into different arrays indexed by the fund id, which later will be used to combine the 
fund characteristic data by fund id'''

T_Bill_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'T_Bill')
T_Bill_df = T_Bill_df.ffill().bfill()

AUM_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'AUM_Filled')
AUM_df = AUM_df.ffill().bfill()

downside_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'downside')
downside_df = downside_df.ffill().bfill()

upside_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'upside')
upside_df = upside_df.ffill().bfill()

minimuminvestment_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'minimuminvestment')
minimuminvestment_df = minimuminvestment_df.ffill().bfill()

managementfee_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'managementfee')
managementfee_df = managementfee_df.ffill().bfill()

incentivefee_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'incentivefee')
incentivefee_df = incentivefee_df.ffill().bfill()

highwatermark_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'highwatermark')
highwatermark_df = highwatermark_df.ffill().bfill()

leveraged_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'leveraged')
leveraged_df = leveraged_df.ffill().bfill()

maxleverage_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'maxleverage')
maxleverage_df = maxleverage_df.ffill().bfill()

avgleverage_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'avgleverage')
avgleverage_df = avgleverage_df.ffill().bfill()

personalcapital_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'personalcapital')
personalcapital_df = personalcapital_df.ffill().bfill()

personalcapitalamount_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'personalcapitalamount')
personalcapitalamount_df = personalcapitalamount_df.ffill().bfill()

openended_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'openended')
openended_df = openended_df.ffill().bfill()

opentopublic_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'opentopublic')
opentopublic_df = opentopublic_df.ffill().bfill()

frequency_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'frequency')
frequency_df = frequency_df.ffill().bfill()

redemptionnoticeperiod_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'redemptionnoticeperiod')
redemptionnoticeperiod_df = redemptionnoticeperiod_df.ffill().bfill()

lockupperiod_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'lockupperiod')
lockupperiod_df = lockupperiod_df.ffill().bfill()

style_indicator_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'style_indicator')
style_indicator_df = style_indicator_df.ffill().bfill()

primarycategory_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'primarycategory')
primarycategory_df = primarycategory_df.ffill().bfill()

ConvArb_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'ConvArb')
ConvArb_df = ConvArb_df.fillna(0)

DedShort_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'DedShort')
DedShort_df = DedShort_df.fillna(0)

EmergingM_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'EmergingM')
EmergingM_df = EmergingM_df.fillna(0)

EquityMarketN_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'EquityMarketN')
EquityMarketN_df = EquityMarketN_df.fillna(0)

EventD_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'EventD')
EventD_df = EventD_df.fillna(0)

FixedIncArb_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'FixedIncArb')
FixedIncArb_df = FixedIncArb_df.fillna(0)

beta_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'beta')
beta_df = beta_df.fillna(0)

GlobalM_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'GlobalM')
GlobalM_df = GlobalM_df.fillna(0)

LongShortEq_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'LongShortEq')
LongShortEq_df = LongShortEq_df.fillna(0)

ManagedFut_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'ManagedFut')
ManagedFut_df = ManagedFut_df.fillna(0)

Other_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'Other')
Other_df = Other_df.fillna(0)

backfill_dummy_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'backfill_dummy')
backfill_dummy_df = backfill_dummy_df.ffill().bfill()

alive_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'alive')
alive_df = alive_df.ffill().bfill()

dropreason_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'dropreason')
dropreason_df = dropreason_df.fillna('-')

inceptiondate_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'inceptiondate')
inceptiondate_df = inceptiondate_df.fillna(0)

dateaddedtotass_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'dateaddedtotass')
dateaddedtotass_df = dateaddedtotass_df.ffill().bfill()

performancestartdate_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'performancestartdate')
performancestartdate_df = performancestartdate_df.ffill().bfill()

performanceenddate_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'performanceenddate')
performanceenddate_df = performanceenddate_df.ffill().bfill()

datedropped_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'datedropped')
datedropped_df = datedropped_df.ffill().bfill()

style_return_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'style_return')
style_return_df = style_return_df.ffill().bfill()

st_adj_rateofreturn_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'st_adj_rateofreturn')
st_adj_rateofreturn_df = st_adj_rateofreturn_df.ffill().bfill()

DeltaMgr_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'DeltaMgr')
DeltaMgr_df = DeltaMgr_df.ffill().bfill()

DeltaOwn_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'DeltaOwn')
DeltaOwn_df = DeltaOwn_df.ffill().bfill()

TotalDelta_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'TotalDelta')
TotalDelta_df = TotalDelta_df.ffill().bfill()

TotalDelta_Winso_01_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'TotalDelta_Winso_01')
TotalDelta_Winso_01_df = TotalDelta_Winso_01_df.ffill().bfill()

Netcashflow_df = TASS.pivot(index = 'refdate', columns = 'ref', values = 'Net cash flow')
Netcashflow_df = Netcashflow_df.fillna(0)

C:\Users\yukta\AppData\Local\Temp\ipykernel_33908\1578932948.py:111: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dateaddedtotass_df = dateaddedtotass_df.ffill().bfill()
C:\Users\yukta\AppData\Local\Temp\ipykernel_33908\1578932948.py:121: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datedropped_df = datedropped_df.ffill().bfill()


In [5]:
'''The fund characteristics indexed by fund id above are combined into different dataframes, each specific to a fund, and stored in a dictionary 
with fund id as the key. Furthermore, the characteristic variables are converted to numeric format using one hot encoding before adding to the database'''
fund_dfs1 = {}

for i in returns_df.columns:
    dfx = pd.DataFrame()
    dfx[f'{i}_return'] = returns_df[i]
    dfx[f'{i}_sma5'] = returns_df[i].rolling(window=5).mean()
    dfx[f'{i}_sma9'] = returns_df[i].rolling(window=9).mean()
    dfx[f'{i}_sma17'] = returns_df[i].rolling(window=17).mean()
    dfx[f'{i}_downside'] = downside_df[i]
    dfx[f'{i}_upside'] = upside_df[i]
    dfx[f'{i}_mgmtfee'] = managementfee_df[i]
    dfx[f'{i}_lvrgdfee'] = leveraged_df[i]
    dfx[f'{i}_inctfee'] = incentivefee_df[i]
    dfx[f'{i}_hwm'] = highwatermark_df[i]
    dfx[f'{i}_lockuperiod'] = lockupperiod_df[i]
    dfx[f'{i}_AUM'] = AUM_df[i]
    dfx[f'{i}_tbill'] = T_Bill_df[i]
    dfx[f'{i}_mininv'] = minimuminvestment_df[i]
    dfx[f'{i}_maxlev'] = maxleverage_df[i]
    dfx[f'{i}_avglev'] = avgleverage_df[i]
    dfx[f'{i}_pcapamt'] = personalcapitalamount_df[i]
    dfx[f'{i}_openended'] = openended_df[i]
    dfx[f'{i}_opentopublic'] = opentopublic_df[i]
    dfx[f'{i}_frequency'] = frequency_df[i]
    dfx[f'{i}_redmnotcprd'] = redemptionnoticeperiod_df[i]
    dfx[f'{i}_stylind'] = style_indicator_df[i]
    dfx[f'{i}_ConvArb'] = ConvArb_df[i]
    dfx[f'{i}_DedShort'] = DedShort_df[i]
    dfx[f'{i}_EmergingM'] = EmergingM_df[i]
    dfx[f'{i}_EquityMarketN'] = EquityMarketN_df[i]
    dfx[f'{i}_EventD'] = EventD_df[i]
    dfx[f'{i}_FixedIncArb'] = FixedIncArb_df[i]
    dfx[f'{i}_GlobalM'] = GlobalM_df[i]
    dfx[f'{i}_LongShortEq'] = LongShortEq_df[i]
    dfx[f'{i}_ManagedFut'] = ManagedFut_df[i]
    dfx[f'{i}_Other'] = Other_df[i]
    dfx[f'{i}_beta'] = beta_df[i]
    dfx[f'{i}_backfill'] = backfill_dummy_df[i]
    dfx[f'{i}_alive'] = alive_df[i]
    
    # One-hot encode only 'dropreason'
    dropreason_col = f'{i}_dropreason'
    dfx[dropreason_col] = dropreason_df[i]
    dfx = pd.get_dummies(dfx, columns=[dropreason_col], prefix=dropreason_col, drop_first=True)
    
    # Convert date columns to numeric (timestamp)
    date_cols = {
        f'{i}_inceptiondate': inceptiondate_df[i],
        f'{i}_dateaddedtotass': dateaddedtotass_df[i],
        f'{i}_performancestartdate': performancestartdate_df[i],
        f'{i}_performanceenddate': performanceenddate_df[i],
        f'{i}_datedropped': datedropped_df[i]
    }

    for col_name, series in date_cols.items():
        dfx[col_name] = pd.to_datetime(series, errors='coerce').astype('int64') // 10**9  # Unix timestamp in seconds

   
    dfx[f'{i}_NCF'] = Netcashflow_df[i]

    dfx.index = pd.to_datetime(returns_df.index)

    fund_dfs1[i] = dfx


In [6]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler

pls_results = {}

for fund_id, df in fund_dfs1.items():
    df = df.dropna()
    if df.shape[0] < 24:  # Ensure enough data
        continue

    # Target variable
    y = df[f'{fund_id}_return']
    X = df.drop(columns=[f'{fund_id}_return'])

    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Fit PLS model (e.g., 5 components)
    pls = PLSRegression(n_components=9)
    pls.fit(X_scaled, y)

    # Store weights (importance) of original features
    feature_weights = pd.Series(np.abs(pls.coef_.flatten()), index=X.columns)
    sorted_weights = feature_weights.sort_values(ascending=False)

    pls_results[fund_id] = sorted_weights
#Now pls_results[fund_id] contains the most predictive features based on PLS coefficients.

C:\Users\yukta\miniconda3\Lib\site-packages\sklearn\cross_decomposition\_pls.py:348: UserWarning: y residual is constant at iteration 7
  warnings.warn(f"y residual is constant at iteration {k}")
C:\Users\yukta\miniconda3\Lib\site-packages\sklearn\cross_decomposition\_pls.py:348: UserWarning: y residual is constant at iteration 7
  warnings.warn(f"y residual is constant at iteration {k}")


In [7]:
# Build filtered dictionary with relevant features
fund_dfs2 = {}

for fund_id, df in fund_dfs1.items():
    df = df.copy()
    df.index = pd.to_datetime(df.index)

    return_col = f"{fund_id}_return"
    if return_col not in df.columns:
        continue  # Skip if return column is missing
    base_features = [
    f"{fund_id}_upside", f"{fund_id}_downside", f"{fund_id}_tbill", f"{fund_id}_sma5", f"{fund_id}_sma9",
    f"{fund_id}_sma17", f"{fund_id}_NCF", f"{fund_id}_AUM"
]
    # Include return column in relevant features
    features_to_keep = [col for col in base_features if col in df.columns]
    features_to_keep.append(return_col)

    # Filter and drop NaNs
    dfx = df[features_to_keep].dropna()
    if not dfx.empty:
        fund_dfs2[fund_id] = dfx


In [8]:
# Joblib library employed for parallel processing of large data, about 4500 dataframes, efficiently
# Ridge regression and Random forest
from joblib import Parallel, delayed
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

def forecast_for_fund(fund_id, df):
    models = {
        'ridge': Ridge(alpha=1.0),
        'random_forest': RandomForestRegressor(n_estimators=100, max_depth=5),
    }

    df = df.sort_index()
    # Locate the return column dynamically
    return_col = [col for col in df.columns if col.endswith('_return')]
    if not return_col:
        return []

    df = df.rename(columns={return_col[0]: 'return'}).dropna()


    df = df.rename(columns={return_col[0]: 'return'}).dropna()
    if df.shape[0] < 24: #Filtering to exclude funds have less than 24 datapoints
        return []

    features = df.drop(columns=['return'])
    target = df['return']
    results = []

    for test_date in pd.date_range(start='2005-01-01', end='2015-12-31', freq='Q'):
        train_data = df.loc['1993-01-01':test_date - pd.DateOffset(months=3)]
        if train_data.shape[0] < 36 or test_date not in df.index:
            continue

        X_train = train_data.drop(columns=['return'])
        y_train = train_data['return']
        X_test = features.loc[[test_date]]
        y_test = target.loc[[test_date]]

        for model_name, model in models.items():
            pipe = Pipeline([('scaler', StandardScaler()), ('model', model)])
            pipe.fit(X_train, y_train)
            pred = pipe.predict(X_test)

            results.append({
                'fund_id': fund_id,
                'model': model_name,
                'test_date': test_date,
                'actual': y_test.values[0],
                'predicted': pred[0],
                'mse': mean_squared_error(y_test, [pred[0]])
            })

    return results

# Parallel processing
from joblib import Parallel, delayed
from tqdm import tqdm
def rolling_forecast_2005_2015_parallel(fund_data):
    all_results = Parallel(n_jobs=2)(  # Use all available CPU cores
        delayed(forecast_for_fund)(fund_id, df) for fund_id, df in tqdm(fund_data.items())
    )
    # Flatten the list of lists
    return pd.DataFrame([res for sublist in all_results for res in sublist])


In [9]:
# Forecast on the test period of 2005 to 2015
forecast_df = rolling_forecast_2005_2015_parallel(fund_dfs2)
# Filter only the rows from the year 2005 for evaluation
preds_2005 = forecast_df[forecast_df['test_date'].dt.year == 2005]

# Display top 5 predictions for 2005
print(preds_2005)

100%|██████████████████████████████████████████████████████████████████████| 3336/3336 [1:09:42<00:00,  1.25s/it]


       fund_id          model  test_date  actual  predicted           mse
0           26          ridge 2005-03-31 -0.0067  -0.001650  2.550103e-05
1           26  random_forest 2005-03-31 -0.0067  -0.009050  5.523163e-06
2           26          ridge 2005-06-30  0.0149   0.013645  1.574827e-06
3           26  random_forest 2005-06-30  0.0149   0.014287  3.757177e-07
4           26          ridge 2005-09-30  0.0192   0.018806  1.549501e-07
...        ...            ...        ...     ...        ...           ...
60227   105138  random_forest 2005-06-30  0.0001   0.000180  6.379753e-09
60228   105138          ridge 2005-09-30 -0.0102  -0.005608  2.108578e-05
60229   105138  random_forest 2005-09-30 -0.0102  -0.008898  1.695676e-06
60230   105138          ridge 2005-12-31 -0.0075  -0.003650  1.482277e-05
60231   105138  random_forest 2005-12-31 -0.0075  -0.008561  1.126465e-06

[5394 rows x 6 columns]


In [10]:
#Lasso and Elastic Net
def forecast_for_fund(fund_id, df):
    models = {
        'lasso': Lasso(alpha=0.01),
        'elasticnet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    }

    df = df.sort_index()
    return_col = [col for col in df.columns if 'return' in col.lower()]
    if not return_col:
        return []

    df = df.rename(columns={return_col[0]: 'return'}).dropna()
    if df.shape[0] < 24:
        return []

    features = df.drop(columns=['return'])
    target = df['return']
    results = []

    for test_date in pd.date_range(start='2005-01-01', end='2015-12-31', freq='ME'):
        train_data = df.loc['1993-01-01':test_date - pd.DateOffset(months=1)]
        if train_data.shape[0] < 36 or test_date not in df.index:
            continue

        X_train = train_data.drop(columns=['return'])
        y_train = train_data['return']
        X_test = features.loc[[test_date]]
        y_test = target.loc[[test_date]]

        for model_name, model in models.items():
            pipe = Pipeline([('scaler', StandardScaler()), ('model', model)])
            pipe.fit(X_train, y_train)
            pred = pipe.predict(X_test)

            results.append({
                'fund_id': fund_id,
                'model': model_name,
                'test_date': test_date,
                'actual': y_test.values[0],
                'predicted': pred[0],
                'mse': mean_squared_error(y_test, [pred[0]])
            })

    return results

# Parallel processing
from joblib import Parallel, delayed

def rolling_forecast_2005_2015_parallel(fund_data):
    all_results = Parallel(n_jobs=2)(  # Use all available CPU cores
        delayed(forecast_for_fund)(fund_id, df) for fund_id, df in tqdm(fund_data.items())
    )
    # Flatten the list of lists
    return pd.DataFrame([res for sublist in all_results for res in sublist])


In [11]:
forecast_df = rolling_forecast_2005_2015_parallel(fund_dfs2)
# Filter only the rows from the year 2005
preds1_2005 = forecast_df[forecast_df['test_date'].dt.year == 2005]

# Display top 5 predictions for 2005
print(preds1_2005)

100%|████████████████████████████████████████████████████████████████████████| 3336/3336 [10:06<00:00,  5.50it/s]


        fund_id       model  test_date  actual  predicted           mse
0            26       lasso 2005-01-31 -0.0132   0.006691  3.956443e-04
1            26  elasticnet 2005-01-31 -0.0132   0.009275  5.051371e-04
2            26       lasso 2005-02-28 -0.0035   0.007106  1.124824e-04
3            26  elasticnet 2005-02-28 -0.0035   0.009275  1.632071e-04
4            26       lasso 2005-03-31 -0.0067   0.006644  1.780504e-04
...         ...         ...        ...     ...        ...           ...
187443   105138  elasticnet 2005-10-31  0.0010   0.001535  2.862945e-07
187444   105138       lasso 2005-11-30  0.0153   0.002304  1.688989e-04
187445   105138  elasticnet 2005-11-30  0.0153   0.001535  1.894734e-04
187446   105138       lasso 2005-12-31 -0.0075   0.001286  7.718784e-05
187447   105138  elasticnet 2005-12-31 -0.0075   0.001703  8.468659e-05

[16610 rows x 6 columns]


In [12]:
# Decision Tree, Gradient Boosting, Deep Neural Network
def forecast_for_fund(fund_id, df):
    models = {
        'decision_tree': DecisionTreeRegressor(max_depth=5),
        'gradient_boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1),
        'dnn': MLPRegressor(hidden_layer_sizes=(50, 20), max_iter=500)
    }

    df = df.sort_index()
    return_col = [col for col in df.columns if 'return' in col.lower()]
    if not return_col:
        return []

    df = df.rename(columns={return_col[0]: 'return'}).dropna()
    if df.shape[0] < 24:
        return []

    features = df.drop(columns=['return'])
    target = df['return']
    results = []

    for test_date in pd.date_range(start='2005-01-01', end='2015-12-31', freq='ME'):
        train_data = df.loc['1993-01-01':test_date - pd.DateOffset(months=1)]
        
        if train_data.shape[0] < 36 or test_date not in df.index:
            continue

        X_train = train_data.drop(columns=['return'])
        y_train = train_data['return']
        X_test = features.loc[[test_date]]
        y_test = target.loc[[test_date]]

        for model_name, model in models.items():
            pipe = Pipeline([('scaler', StandardScaler()), ('model', model)])
            pipe.fit(X_train, y_train)
            pred = pipe.predict(X_test)

            results.append({
                'fund_id': fund_id,
                'model': model_name,
                'test_date': test_date,
                'actual': y_test.values[0],
                'predicted': pred[0],
                'mse': mean_squared_error(y_test, [pred[0]])
            })

    return results

# Parallel processing
from joblib import Parallel, delayed

def rolling_forecast_2005_2015_parallel(fund_data):
    all_results = Parallel(n_jobs=2)(  # Use all available CPU cores
        delayed(forecast_for_fund)(fund_id, df) for fund_id, df in tqdm(fund_data.items())
    )
    # Flatten the list of lists
    return pd.DataFrame([res for sublist in all_results for res in sublist])


In [13]:
forecast_df = rolling_forecast_2005_2015_parallel(fund_dfs2)
# Filter only the rows from the year 2005
preds2_2005 = forecast_df[forecast_df['test_date'].dt.year == 2005]

# Display top 5 predictions for 2005
print(preds2_2005)

100%|██████████████████████████████████████████████████████████████████████| 3336/3336 [1:28:23<00:00,  1.59s/it]


        fund_id              model  test_date  actual  predicted           mse
0            26      decision_tree 2005-01-31 -0.0132  -0.008330  2.371691e-05
1            26  gradient_boosting 2005-01-31 -0.0132  -0.013018  3.309711e-08
2            26                dnn 2005-01-31 -0.0132   0.003908  2.926864e-04
3            26      decision_tree 2005-02-28 -0.0035   0.001575  2.575867e-05
4            26  gradient_boosting 2005-02-28 -0.0035  -0.000454  9.280692e-06
...         ...                ...        ...     ...        ...           ...
281167   105138  gradient_boosting 2005-11-30  0.0153   0.013304  3.985605e-06
281168   105138                dnn 2005-11-30  0.0153   0.101342  7.403194e-03
281169   105138      decision_tree 2005-12-31 -0.0075  -0.009200  2.890000e-06
281170   105138  gradient_boosting 2005-12-31 -0.0075  -0.008142  4.121403e-07
281171   105138                dnn 2005-12-31 -0.0075   0.050662  3.382776e-03

[24915 rows x 6 columns]


In [14]:
#Compare prediction accracy of all models with measn squared error
print(preds_2005[preds_2005['model'] == 'ridge']['mse'].sum())
print(preds_2005[preds_2005['model'] =='random_forest']['mse'].sum())
print(preds1_2005[preds1_2005['model'] =='elasticnet']['mse'].sum())
print(preds1_2005[preds1_2005['model'] =='lasso']['mse'].sum())
print(preds2_2005[preds2_2005['model'] =='decision_tree']['mse'].sum())
print(preds2_2005[preds2_2005['model'] =='gradient_boosting']['mse'].sum())
print(preds2_2005[preds2_2005['model'] =='dnn']['mse'].sum())

1.0124560909358782
0.13656905551534448
6.781944984780712
1.7989703799823331
1.027948829829629
0.409248966383943
1.3881530103178536e+16


In [15]:
#Finalizing on the Gradient Boosting algorithm as our final model due to processing efficiency and prediction accuracy
final_preds = forecast_df[forecast_df['model'] == 'gradient_boosting']

In [16]:
#Number of unique funds after filtering is 1866
list = pd.Series(forecast_df['fund_id'].unique())
list.shape

(1866,)

In [17]:
final_preds.head()
pred_ret_funds = final_preds.pivot(index = 'test_date', columns = 'fund_id', values = 'predicted')
pred_ret_funds = pred_ret_funds.fillna(0)

In [18]:
#Creating a copy of baseline predictions
baseline_q = pred_ret_funds.copy()     

pred_ret_funds.isna().sum()


fund_id
26        0
31        0
35        0
61        0
78        0
         ..
105116    0
105136    0
105138    0
105709    0
105785    0
Length: 1866, dtype: int64

In [19]:
tbill = TASS[['refdate', 'T_Bill']].copy()  # Safe copy
tbill['refdate'] = pd.to_datetime(tbill['refdate'])  # No warning now
tbill.set_index('refdate', inplace=True)
tbill_q = tbill.loc[pred_ret_funds.index, 'T_Bill']


In [21]:
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_checker import check_env

# ======== Custom Portfolio Environment ========
class PortfolioEnv(gym.Env):
    def __init__(self, predicted_returns, baseline_returns, tbill_rate,cov_matrix, window_size=1):
        super().__init__()
        self.predicted_returns = predicted_returns.sort_index()
        self.baseline_returns = baseline_returns.sort_index()
        self.tbill_rate = tbill_rate.sort_index()
        self.cov_matrix = cov_matrix
        self.window_size = window_size
        self.funds = self.predicted_returns.columns.tolist()
        self.n_funds = len(self.funds)
        self.periods = self.predicted_returns.index
        self.current_step = 0

        # Observation space: predicted returns for each fund
        max_abs_val = np.max(np.abs(self.predicted_returns.values))
        self.observation_space = spaces.Box(low=-max_abs_val, high=max_abs_val, shape=(self.n_funds,), dtype=np.float32)

        # Action space: portfolio weights between [0, 1], normalized later
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.n_funds,), dtype=np.float32)


    def reset(self, seed=0, options=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.prev_weights = np.ones(self.n_funds) / self.n_funds
        obs = self.predicted_returns.iloc[self.current_step].values.astype(np.float32)
        obs = np.clip(obs, self.observation_space.low, self.observation_space.high)
        return obs, {}

    def step(self, action):
        
        action = np.array(action)
        abs_sum = np.sum(np.abs(action))
        if abs_sum < 1e-8:
            weights = np.zeros_like(action)
        else:
            weights = action / abs_sum

    
        # Compute return
        ret = np.dot(weights, self.predicted_returns.iloc[self.current_step].values)
        base_ret = np.dot(self.prev_weights, self.baseline_returns.iloc[self.current_step].values)
    
        excess_ret = ret - self.tbill_rate.iloc[self.current_step]
        base_excess = base_ret - self.tbill_rate.iloc[self.current_step]
    
        # Risk penalty term (portfolio variance)
        # cov_matrix is prepared and aligned with fund order
        risk_penalty_factor = 0.01  # Tune hyperparameter
    
        portfolio_variance = weights.T @ self.cov_matrix @ weights
        risk_penalty = risk_penalty_factor * portfolio_variance
    
        # Transaction cost
        transaction_cost_rate = 0.0005
        turnover = np.sum(np.abs(weights - self.prev_weights))
        transaction_cost = transaction_cost_rate * turnover
    
        # Reward: excess return minus baseline excess return, minus transaction cost and risk penalty
        reward = excess_ret - base_excess - transaction_cost - risk_penalty
    
        self.prev_weights = weights
        self.current_step += 1
        done = self.current_step >= len(self.predicted_returns)
    
        if not done:
            obs = self.predicted_returns.iloc[self.current_step].values.astype(np.float32)
            obs = np.clip(obs, self.observation_space.low, self.observation_space.high)
        else:
            obs = np.zeros(self.n_funds, dtype=np.float32)
    
        return obs, reward, done, False, {}


# ======== Load and Prepare Data ========
# baseline_returns, predicted_returns, tbill_rate are pandas DataFrames/Series aligned on index
# Compute covariance matrix for baseline returns
cov_matrix = baseline_q.cov().loc[pred_ret_funds.columns, pred_ret_funds.columns].values

# Create environment instance with covariance matrix
env = PortfolioEnv(pred_ret_funds, baseline_q, tbill_q, cov_matrix)

# ======== Instantiate and Check Env ========
raw_env = PortfolioEnv(pred_ret_funds, baseline_q, tbill_q, cov_matrix)
check_env(raw_env)

# ======== Create Vectorized and Normalized Env ========
vec_env = DummyVecEnv([lambda: PortfolioEnv(pred_ret_funds, baseline_q, tbill_q, cov_matrix)])

vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)

# ======== Train PPO Agent ========
model = PPO(
    "MlpPolicy",
    vec_env,
    learning_rate=5e-5,
    verbose=1,
    tensorboard_log="./ppo_logs/"
)

model.learn(total_timesteps=10000)

# ======== Save Trained Model and Normalization Stats ========
model.save("ppo_portfolio_model")
vec_env.save("vec_normalize.pkl")


Using cpu device
Logging to ./ppo_logs/PPO_7
-----------------------------
| time/              |      |
|    fps             | 225  |
|    iterations      | 1    |
|    time_elapsed    | 9    |
|    total_timesteps | 2048 |
-----------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 198       |
|    iterations           | 2         |
|    time_elapsed         | 20        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 1.9563425 |
|    clip_fraction        | 0.841     |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.65e+03 |
|    explained_variance   | -3.25     |
|    learning_rate        | 5e-05     |
|    loss                 | -0.0818   |
|    n_updates            | 10        |
|    policy_gradient_loss | -0.138    |
|    std                  | 1         |
|    value_loss           | 0.187     |
-------------------------

In [22]:
# Final weight allocation to different funds with reinforcement learning with reward as Sharpe Ratio, model created above
# Allocation output as an excel file
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Recreate env and load normalization stats
test_env = DummyVecEnv([lambda: PortfolioEnv(pred_ret_funds, baseline_q, tbill_q, cov_matrix)])
test_env = VecNormalize.load("vec_normalize.pkl", test_env)
test_env.training = False
test_env.norm_reward = False

# Load model
model = PPO.load("ppo_portfolio_model")

# Evaluate
obs, _ = env.reset()
done = False

portfolio_history = []
dates = env.periods[env.window_size:] 

while not done:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)

    # Convert action to final weights as used in the environment
    weights_raw = (action + 1) / 2
    weights = weights_raw / np.sum(weights_raw) if np.sum(weights_raw) > 1e-8 else np.ones(env.n_funds) / env.n_funds
    
    # Store the current date and weights
    step_index = env.current_step - 1  # Because we incremented after step
    if step_index < len(dates):
        date = dates[step_index]
        portfolio_history.append((date, weights))
# Convert to DataFrame
df_weights = pd.DataFrame(
    [w for d, w in portfolio_history],
    index=[d for d, w in portfolio_history],
    columns=env.funds
)

# Save or inspect
df_weights.index.name = "Date"
df_weights.to_csv("final_portfolios_2005_2015.csv")
print(df_weights.head())



              26        31        35        61        78        86      \
Date                                                                     
2005-03-31  0.001068  0.000560  0.000324  0.001068  0.001068  0.000569   
2005-04-30  0.000923  0.000798  0.000571  0.000622  0.000618  0.001070   
2005-05-31  0.000980  0.000540  0.000529  0.000276  0.000943  0.000678   
2005-06-30  0.000000  0.000406  0.000858  0.000499  0.000300  0.000410   
2005-07-31  0.000000  0.000023  0.000880  0.000140  0.000437  0.000487   

              87        151       167       184     ...    104216    104252  \
Date                                                ...                       
2005-03-31  0.001068  0.001068  0.000990  0.001068  ...  0.000864  0.000819   
2005-04-30  0.000922  0.000771  0.000237  0.001070  ...  0.000000  0.000339   
2005-05-31  0.001057  0.000418  0.000743  0.000256  ...  0.000181  0.000280   
2005-06-30  0.000225  0.000837  0.001071  0.000790  ...  0.000000  0.000889   
2005-07